In [1]:
import numpy as np
import scipy.integrate
import pandas as pd

In [27]:
system_a = -0.3
system_b = 0.1
system_c = -0.1
koopman_operator = np.array([[system_a, 0, 0], [system_b, system_c, -system_b], [0, 0, 2*system_a]])

def koopman_evolve(t, y):
    return np.matmul(koopman_operator, y)

#Idealized phi and phi_inv, gives an upper bound on accuracy of 
#nonlinear network
def ideal_compression(state):
    return np.array([state[0], state[1]])

def ideal_decompression(state):
    return np.array([state[0], state[1], state[0]*state[0]])

#Generator which pre-applies perfect compression and decompression on system
def generate_ideal_system_data(batch_size = 128, t_span=1.0):
    systems = np.empty([batch_size, ANTIKOOPMAN_DIMENSION])
    initial_conditions = np.empty([batch_size, ANTIKOOPMAN_DIMENSION])
    while True:
        for i in range(batch_size):
            y0 = np.empty([STATE_DIMENSION])
            y0[0], y0[1] = np.random.random(2)
            y0[2] = y0[0]*y0[0] 
            #y0.reshape(3,1)
            evolution = scipy.integrate.solve_ivp(koopman_evolve,(0,t_span), y0).y[:,-1]
            initial_conditions[i] = ideal_compression(y0)
            systems[i] = ideal_compression(evolution)
        yield(initial_conditions, systems)
        
'''
def generate_dynamical_system_evolution(y0, samples = 100, t_step = 1.0):
    evolution = np.empty([samples, len(y0)])
    evolution[0] = np.array(y0)
    for i in range(1, samples):
        #koopman_evolve independent of t, so any time interval of t_step should work just as well
        evolution[i] = scipy.integrate.solve_ivp(koopman_evolve,(t_step*(i-1), t_step*i), evolution[i-1]).y[:,-1]
    return evolution
'''

def generate_dynamical_system_evolution(y0,t_range=40.,delta_t=.01):
    evolution = np.empty([int(t_range/delta_t),len(y0)])
    #evolution[0] = np.array(y0)
    evolved = scipy.integrate.solve_ivp(koopman_evolve, (0.,t_range), y0, t_eval=np.linspace(0.,t_range,int(t_range/delta_t)), rtol=1e-11, atol=1e-12)
    for i in range(len(evolved.t)):
        evolution[i] = evolved.y[:,i]
    return evolution

In [6]:
evolution = generate_dynamical_system_evolution([1.,1.,1.],1000, 40.)

In [7]:
pd.DataFrame(evolution).to_csv('./evolutions/111_t40_1000.csv')

In [21]:
evolution

,0,1,2
0,1.000000e+00,1.000000,1.000000e+00
1,1.020201e+00,1.079205,9.801987e-01
2,1.040811e+00,1.156842,9.607894e-01
3,1.061837e+00,1.232942,9.417645e-01
4,1.083287e+00,1.307535,9.231163e-01
...,...,...,...
995,4.389956e+08,5.000000,2.277927e-09
996,4.478639e+08,5.000000,2.232821e-09
997,4.569114e+08,5.000000,2.188608e-09
998,4.661416e+08,5.000000,2.145271e-09


In [25]:
square = 5

for i in range(1,square+1):
    for k in range(1,square+1):
        evolution = generate_dynamical_system_evolution([i, k, 1./i], samples=4000, t_step=0.05)
        pd.DataFrame(evolution).to_csv('./evolutions/evolution{}.csv'.format(square*(i-1)+(k-1)))

In [29]:
for i in range(50):
    y0 = np.empty([3])
    y0[0], y0[1] = np.random.random(2)
    y0[2] = y0[0]*y0[0]
    evolution = generate_dynamical_system_evolution(y0, t_range=20.,delta_t=.01)
    pd.DataFrame(evolution).to_csv('./evolutions/50randscap1_dt0p01/evolution{}.csv'.format(i))